<a href="https://colab.research.google.com/github/prasadoff12/Listening-Speaking/blob/main/Listening_(FACE2FACE_AI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#LISTENING MODEL

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import random
from sklearn import preprocessing

from google.colab import files
import io

In [ ]:
#Steps taken video converted in to (speaking & listening) CSV file.
# 1. Videos has been converted in to CSV file by using OPENFACE
# 2. After converted CSV file from video, this CSV file contain two face_id's. Since video has two persons conversation.
# 3. We extracted this CSV file as speaking and listening csv files by using face_id's. Note:  face_id '0' as speaker
     # and face_id '1' as listener. 
# 4.Because in listening section we take SEQ2SEQ model for training.
# 5.This listening model takes the AU+POSE of SPEAKER  as encoder input while AU+POSE of LISTENER as decoder input and
    # output AU+POSE of LISTENER
  #Check in POC 3.2 section for more details

##Uploaded Speaking CSV file

In [ ]:
X1="/content/drive/MyDrive/Speaking.csv"

In [ ]:
X1 = pd.read_csv(X1)
X1.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,p_29,p_30,p_31,p_32,p_33,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,-0.099604,0.260552,-0.960308,-0.251023,0.256564,-0.933361,-0.183,0.267,594.9,596.5,600.8,605.1,606.9,605.5,601.0,596.7,588.9,591.8,595.7,600.2,604.8,608.3,610.9,608.2,604.4,600.1,595.8,591.9,599.0,600.9,602.8,603.5,602.7,600.8,598.9,...,-0.023,-0.039,-0.015,-0.018,0.010,0.0,0.17,0.00,0.0,0.0,0.0,0.0,0.0,0.34,0.71,0.00,1.13,0.0,0.0,0.66,0.82,0.88,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1
1,2,0,0.033,0.98,1,-0.099225,0.275687,-0.956113,-0.249572,0.273478,-0.928937,-0.183,0.283,594.6,596.2,600.3,604.6,606.4,605.1,600.7,596.5,588.8,591.6,595.4,599.8,604.3,607.8,610.3,607.7,604.0,599.8,595.5,591.7,598.7,600.6,602.4,603.1,602.3,600.4,598.6,...,-0.072,-0.156,-0.001,0.006,0.026,0.0,0.47,0.12,0.0,0.0,0.0,0.0,0.0,0.18,0.87,0.06,1.21,0.0,0.0,0.63,1.05,1.27,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1
2,3,0,0.067,0.98,1,-0.103005,0.275334,-0.955814,-0.251534,0.273052,-0.928533,-0.186,0.283,594.7,596.3,600.5,604.7,606.6,605.3,600.8,596.6,588.9,591.7,595.5,600.0,604.4,607.9,610.4,607.8,604.1,599.9,595.6,591.8,598.8,600.7,602.5,603.2,602.4,600.5,598.7,...,-0.080,-0.154,-0.007,0.004,0.033,0.0,0.30,0.09,0.0,0.0,0.0,0.0,0.0,0.17,0.81,0.06,1.04,0.0,0.0,0.70,1.18,1.31,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1
3,4,0,0.100,0.98,1,-0.107448,0.287825,-0.951636,-0.257267,0.285192,-0.923298,-0.192,0.297,594.8,596.4,600.5,604.7,606.5,605.2,600.9,596.7,589.0,591.8,595.6,600.0,604.4,607.8,610.3,607.8,604.1,599.9,595.7,591.9,598.9,600.7,602.5,603.2,602.4,600.6,598.8,...,-0.071,-0.169,-0.019,0.004,0.021,0.0,0.41,0.23,0.0,0.0,0.0,0.0,0.0,0.25,0.62,0.15,0.83,0.0,0.0,0.72,1.29,1.38,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1
4,5,0,0.133,0.98,1,-0.110276,0.287079,-0.951538,-0.259898,0.283941,-0.922947,-0.195,0.296,594.9,596.5,600.6,604.8,606.6,605.3,600.9,596.8,589.1,591.9,595.7,600.1,604.5,607.9,610.4,607.9,604.2,600.0,595.8,592.0,599.0,600.8,602.6,603.3,602.5,600.7,598.9,...,-0.082,-0.161,-0.021,0.015,0.022,0.0,0.44,0.16,0.0,0.0,0.0,0.0,0.0,0.27,0.75,0.34,0.82,0.0,0.0,0.77,1.37,1.46,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1


In [ ]:
X1.shape

(39682, 714)

##Extracted AU+POSE as 20D vector

In [ ]:
#Extracted AU+POSE as 20D vector which comprises of 17-D AU and 3-D head pose as per Documentation. 
#Please check section 3.3

In [ ]:
filter_columns=columns[3:]
filter_column=[i for i in filter_columns if "_r" in i and "p_" not in i or "pose_R" in i]
filter_column
X1=X1[filter_column]
X1

,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,0.228,0.101,-0.080,0.00,0.17,0.00,0.00,0.0,0.00,0.00,0.0,0.34,0.71,0.00,1.13,0.0,0.0,0.66,0.82,0.88
1,0.228,0.108,-0.088,0.00,0.47,0.12,0.00,0.0,0.00,0.00,0.0,0.18,0.87,0.06,1.21,0.0,0.0,0.63,1.05,1.27
2,0.224,0.115,-0.089,0.00,0.30,0.09,0.00,0.0,0.00,0.00,0.0,0.17,0.81,0.06,1.04,0.0,0.0,0.70,1.18,1.31
3,0.237,0.127,-0.095,0.00,0.41,0.23,0.00,0.0,0.00,0.00,0.0,0.25,0.62,0.15,0.83,0.0,0.0,0.72,1.29,1.38
4,0.230,0.132,-0.090,0.00,0.44,0.16,0.00,0.0,0.00,0.00,0.0,0.27,0.75,0.34,0.82,0.0,0.0,0.77,1.37,1.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39677,-0.188,-0.187,-0.091,0.23,0.35,0.82,1.00,0.0,0.00,0.55,0.0,0.00,0.04,0.56,0.00,0.0,0.0,0.00,0.00,0.60
39678,-0.183,-0.187,-0.088,0.40,0.34,0.86,0.68,0.0,0.00,0.54,0.0,0.00,0.22,0.45,0.00,0.0,0.0,0.00,0.00,0.40
39679,-0.185,-0.187,-0.087,0.27,0.35,0.88,0.82,0.0,0.00,0.49,0.0,0.00,0.27,0.37,0.00,0.0,0.0,0.00,0.00,0.48
39680,-0.187,-0.190,-0.087,0.53,0.42,0.78,1.04,0.0,0.00,0.74,0.0,0.00,0.50,0.35,0.00,0.0,0.0,0.00,0.08,0.48


##Preprocessing of speaking model

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X1_Scaled=min_max_scaler.fit_transform(X1)

In [ ]:
X1=pd.DataFrame(columns=filter_column,data=X1_Scaled)

In [ ]:
X1

,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
0,0.517353,0.554335,0.637750,0.000000,0.037199,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.114478,0.147609,0.000000,0.381757,0.0,0.0,0.133603,0.174468,0.220551
1,0.517353,0.558382,0.630124,0.000000,0.102845,0.035398,0.000000,0.0,0.000000,0.000000,0.0,0.060606,0.180873,0.014925,0.408784,0.0,0.0,0.127530,0.223404,0.318296
2,0.515502,0.562428,0.629171,0.000000,0.065646,0.026549,0.000000,0.0,0.000000,0.000000,0.0,0.057239,0.168399,0.014925,0.351351,0.0,0.0,0.141700,0.251064,0.328321
3,0.521518,0.569364,0.623451,0.000000,0.089716,0.067847,0.000000,0.0,0.000000,0.000000,0.0,0.084175,0.128898,0.037313,0.280405,0.0,0.0,0.145749,0.274468,0.345865
4,0.518279,0.572254,0.628217,0.000000,0.096280,0.047198,0.000000,0.0,0.000000,0.000000,0.0,0.090909,0.155925,0.084577,0.277027,0.0,0.0,0.155870,0.291489,0.365915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39677,0.324850,0.387861,0.627264,0.054632,0.076586,0.241888,0.221729,0.0,0.000000,0.221774,0.0,0.000000,0.008316,0.139303,0.000000,0.0,0.0,0.000000,0.000000,0.150376
39678,0.327163,0.387861,0.630124,0.095012,0.074398,0.253687,0.150776,0.0,0.000000,0.217742,0.0,0.000000,0.045738,0.111940,0.000000,0.0,0.0,0.000000,0.000000,0.100251
39679,0.326238,0.387861,0.631077,0.064133,0.076586,0.259587,0.181818,0.0,0.000000,0.197581,0.0,0.000000,0.056133,0.092040,0.000000,0.0,0.0,0.000000,0.000000,0.120301
39680,0.325312,0.386127,0.631077,0.125891,0.091904,0.230088,0.230599,0.0,0.000000,0.298387,0.0,0.000000,0.103950,0.087065,0.000000,0.0,0.0,0.000000,0.017021,0.120301


In [ ]:
X1.describe()

,pose_Rx,pose_Ry,pose_Rz,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r
count,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000,39682.000000
mean,0.458529,0.506503,0.693817,0.163263,0.190953,0.082890,0.119064,0.049657,0.169313,0.062028,0.068781,0.107372,0.085976,0.107370,0.128682,0.154036,0.032056,0.192711,0.196140,0.159741
std,0.060455,0.195304,0.076649,0.153890,0.212685,0.140167,0.134727,0.105222,0.174674,0.126158,0.120826,0.164681,0.113627,0.096137,0.166739,0.172724,0.064012,0.189096,0.181025,0.180068
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.420176,0.320231,0.652050,0.026128,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012438,0.000000,0.000000,0.000000,0.026316,0.025532,0.015038
50%,0.459509,0.436994,0.696854,0.130641,0.124726,0.000000,0.079823,0.000000,0.118257,0.000000,0.000000,0.006734,0.022869,0.092040,0.013514,0.098246,0.000000,0.139676,0.159574,0.112782
75%,0.495141,0.696532,0.743565,0.247031,0.286652,0.126844,0.192905,0.043077,0.288382,0.068548,0.103659,0.168350,0.153846,0.179104,0.253378,0.270175,0.036364,0.303644,0.321277,0.218045
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


###Reshaping and added Batch size speaking model to fit in to Keras model

In [ ]:
X1=np.array(X1)
X1=X1.reshape(1,X1.shape[0],X1.shape[1])

##Uploaded Listening CSV file

In [ ]:
X2="/content/drive/MyDrive/Listening.csv"

In [ ]:
X2 = pd.read_csv(X2)
X2.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,p_29,p_30,p_31,p_32,p_33,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,1,0.000,0.98,1,0.293664,0.324240,-0.899238,0.227117,0.387195,-0.893587,0.283,0.378,193.0,194.2,198.0,202.3,204.6,203.8,199.6,195.3,188.8,190.9,194.3,198.8,203.4,207.2,210.0,207.5,203.7,199.5,195.4,191.7,197.3,199.3,201.0,201.5,200.5,198.6,196.8,...,0.091,-0.192,0.069,0.160,0.131,0.04,0.36,0.40,0.0,0.24,1.44,0.0,0.34,0.46,1.59,0.00,0.00,1.23,0.0,3.18,2.38,0.64,1,0,0,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1
1,2,1,0.033,0.98,1,0.348488,0.384874,-0.854651,0.193425,0.448709,-0.872495,0.304,0.450,193.1,194.4,198.0,201.9,203.7,202.7,198.8,195.0,188.2,190.8,194.1,198.0,201.9,205.0,207.3,205.1,201.8,198.0,194.2,190.9,196.9,198.6,200.2,200.7,199.9,198.2,196.6,...,0.337,-0.209,-0.288,0.108,0.249,0.00,0.41,0.87,0.0,0.48,1.86,0.0,0.64,0.40,1.78,0.00,0.02,1.44,0.0,3.43,2.46,1.47,1,0,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1
2,3,1,0.067,0.98,1,0.370489,0.437746,-0.819217,0.138125,0.473056,-0.870138,0.292,0.494,192.0,193.2,196.8,200.6,202.4,201.4,197.6,193.8,186.1,189.0,192.6,196.6,200.4,203.3,205.4,203.5,200.5,196.7,192.8,189.2,195.5,197.2,198.8,199.3,198.5,196.8,195.3,...,0.206,-0.160,-0.130,0.119,0.274,0.02,0.34,1.31,0.0,0.66,1.43,0.0,0.67,0.54,2.01,0.00,0.00,1.42,0.0,3.53,2.26,1.94,1,0,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1
3,4,1,0.100,0.98,1,0.373134,0.440236,-0.816678,0.139026,0.486719,-0.862425,0.296,0.504,189.9,191.0,194.6,198.4,200.4,199.5,195.7,191.8,184.4,187.1,190.4,194.3,198.1,201.3,203.6,201.6,198.3,194.5,190.7,187.2,193.6,195.3,196.8,197.3,196.5,194.8,193.3,...,0.236,-0.013,-0.043,0.152,0.212,0.04,0.29,1.25,0.0,0.67,1.98,0.0,0.82,0.44,1.74,0.12,0.22,1.21,0.0,3.50,2.26,2.21,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,1,0,1
4,5,1,0.133,0.98,1,0.369415,0.449989,-0.813045,0.132647,0.480344,-0.866991,0.290,0.506,186.8,188.1,191.9,195.8,197.7,196.7,192.6,188.7,181.8,184.2,187.3,191.1,195.2,198.6,201.3,198.8,195.2,191.1,187.4,184.3,190.7,192.5,194.1,194.7,193.9,192.1,190.5,...,0.309,-0.031,0.014,0.140,0.130,0.00,0.27,1.33,0.0,0.66,1.71,0.0,0.95,0.61,1.53,0.12,0.10,0.83,0.0,3.35,1.67,2.44,1,0,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1


##Extracted AU+POSE as 20D vector

In [ ]:
#Extracted AU+POSE as 20D vector which comprises of 17-D AU and 3-D head pose as per Documentation. 
#Please check section 3.3

In [ ]:
filter_columns=columns[3:]
filter_column=[i for i in filter_columns if "_r" in i and "p_" not in i or "pose_R" in i]
filter_column
X2=X2[filter_column]
X2

(41500, 20)

##Preprocessing of listening model

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X2_Scaled=min_max_scaler.fit_transform(X2)

In [ ]:
X2=pd.DataFrame(columns=filter_column1, data=X2_Scaled)

In [ ]:
X2

,pose_Rx,pose_Ry,pose_Rz,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU45_c
0,0.560568,0.388535,0.307979,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,0.572264,0.387261,0.305742,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,0.576441,0.391083,0.299031,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3,0.578112,0.393631,0.294556,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,0.581454,0.407643,0.287845,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,0.401838,0.731210,0.263236,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
41496,0.404344,0.730573,0.260999,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
41497,0.406850,0.729299,0.260254,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
41498,0.408521,0.729299,0.259508,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


###Reshaping and added Batch size listening model to fit in to Keras model

In [ ]:
X2=np.array(X2)
X2=X2.reshape(1,X2.shape[0],X2.shape[1])

#Encoder & Decoder Architecture (SEQ2SEQ MODEL)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from numpy import array

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
    
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
    
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

#Compile and training model

In [ ]:
import sys

n_features=20

#define model
train, infenc, infdec = define_models(n_features, n_features, 128)

In [ ]:
train.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None, 20)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 128), (None, 76288       input_5[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 128),  76288       input_6[0][0]                    
                                                                 lstm_2[0][1]               

In [ ]:
train.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
train.fit([X1,X2],X2,epochs=100)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 23.7957 - accuracy: 0.1086
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 23.6236 - accuracy: 0.1889
Epoch 3/100
1/1 [==============================] - 2s 2s/step - loss: 23.4722 - accuracy: 0.2605
Epoch 4/100
1/1 [==============================] - 2s 2s/step - loss: 23.3414 - accuracy: 0.3288
Epoch 5/100
1/1 [==============================] - 2s 2s/step - loss: 23.2341 - accuracy: 0.3880
Epoch 6/100
1/1 [==============================] - 2s 2s/step - loss: 23.1610 - accuracy: 0.4210
Epoch 7/100
1/1 [==============================] - 2s 2s/step - loss: 23.1434 - accuracy: 0.4527
Epoch 8/100
1/1 [==============================] - 2s 2s/step - loss: 23.2080 - accuracy: 0.4503
Epoch 9/100
1/1 [==============================] - 2s 2s/step - loss: 23.3652 - accuracy: 0.4006
Epoch 10/100
1/1 [==============================] - 2s 2s/step - loss: 23.5854 - accuracy: 0.3655
Epoch 11/100
1/1 [===========

#Testing

In [ ]:
Y1="/content/drive/MyDrive/test0.csv"
Y1=pd.read_csv(Y1)
Y1.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,p_29,p_30,p_31,p_32,p_33,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,-0.269692,0.282291,-0.920640,-0.382298,0.275819,-0.881914,-0.347,0.300,568.1,569.9,574.6,579.4,581.4,579.9,574.8,570.1,562.6,565.7,569.8,574.7,579.7,583.8,586.9,583.9,579.4,574.4,569.7,565.6,572.8,574.9,576.9,577.7,576.8,574.8,572.8,...,-0.226,0.250,0.032,-0.064,0.011,1.90,3.17,0.18,0.16,0.0,1.09,0.0,0.0,0.0,0.0,1.00,0.00,0.0,0.0,1.66,1.84,2.75,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1
1,2,0,0.033,0.98,1,-0.270320,0.290973,-0.917748,-0.373486,0.273734,-0.886328,-0.343,0.303,568.4,570.2,574.7,579.4,581.5,580.0,575.1,570.4,562.7,566.0,570.2,575.2,580.0,583.8,586.6,583.9,579.7,574.9,570.2,565.9,573.0,575.0,576.9,577.7,576.8,574.8,572.9,...,-0.212,0.130,0.006,-0.066,-0.024,1.86,3.06,0.24,0.00,0.0,1.10,0.0,0.0,0.0,0.0,0.99,0.00,0.0,0.0,1.74,2.02,2.83,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1
2,3,0,0.067,0.98,1,-0.293904,0.291239,-0.910385,-0.369904,0.283068,-0.884897,-0.354,0.310,568.8,570.7,575.4,580.2,582.3,580.7,575.6,570.8,563.7,566.9,571.1,576.1,581.1,585.1,588.1,585.1,580.7,575.7,571.0,566.9,573.6,575.6,577.7,578.5,577.6,575.5,573.5,...,-0.201,0.237,0.067,-0.030,-0.034,1.91,3.02,0.24,0.00,0.0,0.82,0.0,0.0,0.0,0.0,0.99,0.00,0.0,0.0,1.83,2.16,2.63,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1
3,4,0,0.100,0.93,1,-0.293145,0.285538,-0.912433,-0.372546,0.276350,-0.885912,-0.355,0.303,569.8,571.7,576.5,581.2,583.3,581.7,576.6,571.8,564.8,567.9,572.0,577.0,582.1,586.2,589.2,586.1,581.5,576.5,571.8,567.8,574.6,576.7,578.7,579.6,578.7,576.6,574.5,...,-0.133,0.203,0.088,-0.026,-0.063,1.95,2.96,0.28,0.06,0.0,1.01,0.0,0.0,0.0,0.0,0.92,0.00,0.0,0.0,1.84,2.11,2.41,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1
4,5,0,0.133,0.98,1,-0.280837,0.267403,-0.921752,-0.344356,0.249958,-0.904953,-0.330,0.276,570.4,572.4,577.3,582.2,584.2,582.6,577.4,572.5,565.1,568.4,572.7,577.8,582.9,586.9,589.7,586.6,582.2,577.2,572.5,568.3,575.2,577.4,579.6,580.5,579.5,577.3,575.2,...,-0.045,0.197,0.082,-0.041,-0.099,1.75,2.71,0.00,0.13,0.0,0.98,0.0,0.0,0.0,0.0,1.15,0.13,0.0,0.0,1.23,1.42,2.28,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1


In [ ]:
Y1.shape

(22026, 714)

In [ ]:
columnss=list(Y1.columns)
print(columnss)

['frame', ' face_id', ' timestamp', ' confidence', ' success', ' gaze_0_x', ' gaze_0_y', ' gaze_0_z', ' gaze_1_x', ' gaze_1_y', ' gaze_1_z', ' gaze_angle_x', ' gaze_angle_y', ' eye_lmk_x_0', ' eye_lmk_x_1', ' eye_lmk_x_2', ' eye_lmk_x_3', ' eye_lmk_x_4', ' eye_lmk_x_5', ' eye_lmk_x_6', ' eye_lmk_x_7', ' eye_lmk_x_8', ' eye_lmk_x_9', ' eye_lmk_x_10', ' eye_lmk_x_11', ' eye_lmk_x_12', ' eye_lmk_x_13', ' eye_lmk_x_14', ' eye_lmk_x_15', ' eye_lmk_x_16', ' eye_lmk_x_17', ' eye_lmk_x_18', ' eye_lmk_x_19', ' eye_lmk_x_20', ' eye_lmk_x_21', ' eye_lmk_x_22', ' eye_lmk_x_23', ' eye_lmk_x_24', ' eye_lmk_x_25', ' eye_lmk_x_26', ' eye_lmk_x_27', ' eye_lmk_x_28', ' eye_lmk_x_29', ' eye_lmk_x_30', ' eye_lmk_x_31', ' eye_lmk_x_32', ' eye_lmk_x_33', ' eye_lmk_x_34', ' eye_lmk_x_35', ' eye_lmk_x_36', ' eye_lmk_x_37', ' eye_lmk_x_38', ' eye_lmk_x_39', ' eye_lmk_x_40', ' eye_lmk_x_41', ' eye_lmk_x_42', ' eye_lmk_x_43', ' eye_lmk_x_44', ' eye_lmk_x_45', ' eye_lmk_x_46', ' eye_lmk_x_47', ' eye_lmk_x_48', ' 

In [ ]:
filter_column1=[i for i in columnss if "_r" in i and "p_" not in i or "pose_R" in i]

In [ ]:
Y1=Y1[filter_column1]
Y1.shape

(22026, 20)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
Y1_Scaled=min_max_scaler.fit_transform(Y1)

In [ ]:
Y1=pd.DataFrame(columns=filter_column1,data=Y1_Scaled)
Y1.head()

,pose_Rx,pose_Ry,pose_Rz,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU45_c
0,0.619788,0.491919,0.401049,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
1,0.619166,0.489394,0.404048,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
2,0.619166,0.486364,0.405547,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
3,0.621655,0.486364,0.407796,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.612943,0.481818,0.413043,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
Y1=np.array(Y1)
Y1=Y1.reshape(1,Y1.shape[0],Y1.shape[1])

In [ ]:
Y2="/content/drive/MyDrive/test1.csv"
Y2=pd.read_csv(Y2)
Y2.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,p_29,p_30,p_31,p_32,p_33,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,1,0.000,0.98,1,0.431574,0.249567,-0.866868,0.269705,0.253778,-0.928900,0.372,0.273,157.3,159.1,163.8,168.8,171.0,169.5,164.5,159.5,148.8,152.8,158.0,163.7,168.9,172.5,174.8,172.3,168.4,163.4,158.1,153.1,161.8,164.0,166.0,166.8,165.8,163.7,161.6,...,-0.616,0.163,0.050,0.075,0.255,0.00,0.51,2.37,0.14,0.0,0.89,0.41,0.0,0.0,0.0,0.71,0.43,0.32,0.14,0.0,0.0,1.70,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,2,1,0.033,0.98,1,0.441327,0.251153,-0.861483,0.264316,0.246356,-0.932441,0.375,0.271,157.2,158.9,163.6,168.6,170.9,169.4,164.5,159.5,148.6,152.5,157.6,163.3,168.4,172.1,174.5,172.0,168.1,163.1,157.9,152.8,161.7,163.9,165.9,166.7,165.7,163.5,161.5,...,-0.647,0.077,0.079,0.087,0.259,0.08,0.68,2.39,0.23,0.0,0.86,0.32,0.0,0.0,0.0,0.69,0.40,0.31,0.15,0.0,0.0,1.74,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,3,1,0.067,0.98,1,0.409948,0.236338,-0.880958,0.314313,0.232779,-0.920338,0.382,0.255,156.8,158.5,163.4,168.7,171.2,169.9,164.6,159.4,149.6,153.1,157.8,163.4,168.8,172.8,175.6,172.7,168.5,163.4,158.2,153.4,161.8,164.1,166.3,167.1,165.9,163.6,161.4,...,-0.497,0.179,0.087,0.105,0.159,0.02,0.51,2.24,0.35,0.0,0.91,0.22,0.0,0.0,0.0,0.70,0.42,0.22,0.24,0.0,0.0,1.20,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,4,1,0.100,0.98,1,0.391938,0.226778,-0.891603,0.314332,0.208987,-0.926024,0.371,0.235,156.2,158.0,163.1,168.5,171.1,169.7,164.2,158.8,149.5,152.9,157.7,163.2,168.7,172.9,175.8,172.8,168.5,163.3,158.1,153.3,161.3,163.8,166.1,166.9,165.7,163.3,161.0,...,-0.540,0.136,0.109,0.111,0.157,0.00,0.42,2.23,0.25,0.0,0.71,0.19,0.0,0.0,0.0,0.71,0.40,0.20,0.20,0.0,0.0,1.02,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,5,1,0.133,0.98,1,0.388519,0.206119,-0.898091,0.334168,0.176674,-0.925807,0.377,0.207,156.2,158.1,163.2,168.6,171.1,169.6,164.1,158.7,150.0,153.2,157.6,163.1,168.7,173.0,176.0,172.8,168.3,163.1,158.0,153.5,161.4,163.9,166.2,167.1,165.9,163.5,161.1,...,-0.552,0.042,0.063,0.079,0.173,0.07,0.51,2.07,0.33,0.0,0.43,0.11,0.0,0.0,0.0,0.65,0.33,0.32,0.28,0.0,0.0,0.73,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
columnss2=list(Y2.columns)

In [ ]:
filter_column2=[i for i in columnss2 if "_r" in i and "p_" not in i or "pose_R" in i]

In [ ]:
Y2=Y2[filter_column2]
Y2.shape

(16602, 20)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
Y2_Scaled=min_max_scaler.fit_transform(Y2)

In [ ]:
Y2=pd.DataFrame(columns=filter_column2,data=Y2_Scaled)
Y2.head()

,pose_Rx,pose_Ry,pose_Rz,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU45_c
0,0.257833,0.329662,0.836314,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.251958,0.326725,0.835230,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.253264,0.325991,0.835772,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.254569,0.325991,0.836314,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.254569,0.325257,0.836856,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
Y2=np.array(Y2)
Y2=Y2.reshape(1,Y2.shape[0],Y2.shape[1])

In [ ]:
Y1_test=[Y1,Y2]
Y2_test=Y2

##Prediction

In [ ]:
listening_predict=train.predict(Y1_test)

In [ ]:
listening_predict

array([[[0.02183246, 0.07639188, 0.0178085 , ..., 0.18539564,
         0.02827101, 0.10533426],
        [0.02254889, 0.08383596, 0.01765936, ..., 0.17468883,
         0.02619199, 0.10919201],
        [0.02265666, 0.08599592, 0.01776174, ..., 0.17269737,
         0.02575534, 0.11066277],
        ...,
        [0.02265602, 0.08775834, 0.01803024, ..., 0.17272477,
         0.02559614, 0.11204228],
        [0.02265237, 0.08775206, 0.01802204, ..., 0.17272264,
         0.02559411, 0.11202785],
        [0.02263106, 0.08762824, 0.01800474, ..., 0.1727558 ,
         0.0256083 , 0.11203333]]], dtype=float32)

In [ ]:
listening_predict.shape

(1, 16602, 20)